<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Target" data-toc-modified-id="Target-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Target</a></span></li></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import seaborn as sb
import lightgbm as lgb
from sklearn.model_selection import train_test_split


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
# # Load data
df_train = pd.read_csv('../data/training_set_VU_DM.csv')

df_test = pd.read_csv('../data/test_set_VU_DM.csv')

df_example_output = pd.read_csv('../data/submission_sample.csv')

<h3>Target</h3>

We need to predict search order. 

test set misses: position, click_bool, booking_bool, gross_booking_usd

each line in the data describes one hotel in one search

In [40]:
searches = df_train['srch_id'].unique()
properties = df_train['prop_id'].unique()
print(len(searches))
print(len(properties))

199795
129113


In [8]:
df_example_output.head(100)

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
95,10,11203
96,10,23777
97,10,31281
98,10,37420


In [26]:
print(df_train.shape)
print(df_test.shape)
df_train.describe()

(4958347, 54)
(4959183, 50)


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
count,4.958347e+06,4.958347e+06,4.958347e+06,251866.000000,252988.000000,4.958347e+06,4.958347e+06,4.958347e+06,4.950983e+06,4.958347e+06,...,96174.000000,315348.000000,356422.000000,138515.000000,1.916654e+06,1.987503e+06,614730.000000,4.958347e+06,138390.000000,4.958347e+06
mean,1.663666e+05,9.953133e+00,1.753405e+02,3.374334,176.022659,1.739739e+02,7.007918e+04,3.180525e+00,3.777777e+00,6.346994e-01,...,17.250473,0.145969,0.083202,19.433267,-6.089936e-02,9.962752e-03,22.430384,4.474858e-02,386.283316,2.791051e-02
std,9.611223e+04,7.646890e+00,6.591625e+01,0.692519,107.254493,6.834525e+01,4.060992e+04,1.051024e+00,1.050329e+00,4.815144e-01,...,31.160313,0.578202,0.316722,54.370221,4.691723e-01,2.029142e-01,895.965854,2.067514e-01,821.190577,1.647165e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.410000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,0.000000e+00,0.000000,0.000000e+00
25%,8.293600e+04,5.000000e+00,1.000000e+02,2.920000,109.810000,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,...,6.000000,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,124.000000,0.000000e+00
50%,1.665070e+05,5.000000e+00,2.190000e+02,3.450000,152.240000,2.190000e+02,6.963800e+04,3.000000e+00,4.000000e+00,1.000000e+00,...,11.000000,0.000000,0.000000,12.000000,0.000000e+00,0.000000e+00,11.000000,0.000000e+00,218.400000,0.000000e+00
75%,2.497240e+05,1.400000e+01,2.190000e+02,3.930000,213.490000,2.190000e+02,1.051680e+05,4.000000e+00,4.500000e+00,1.000000e+00,...,18.000000,1.000000,0.000000,20.000000,0.000000e+00,0.000000e+00,17.000000,0.000000e+00,429.790000,0.000000e+00
max,3.327850e+05,3.400000e+01,2.310000e+02,5.000000,1958.700000,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,...,1620.000000,1.000000,1.000000,9900.000000,1.000000e+00,1.000000e+00,149400.000000,1.000000e+00,159292.380000,1.000000e+00


In [24]:
df_train.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

In [19]:
# iterate over variables
for col in df_train.columns:
    # Describe value column for variable
    print(f"Variable {col}:")
    print(df_train[col].describe())

Variable srch_id:
count    4.958347e+06
mean     1.663666e+05
std      9.611223e+04
min      1.000000e+00
25%      8.293600e+04
50%      1.665070e+05
75%      2.497240e+05
max      3.327850e+05
Name: srch_id, dtype: float64
Variable date_time:
count                 4958347
unique                 198615
top       2012-11-02 14:04:51
freq                       95
Name: date_time, dtype: object
Variable site_id:
count    4.958347e+06
mean     9.953133e+00
std      7.646890e+00
min      1.000000e+00
25%      5.000000e+00
50%      5.000000e+00
75%      1.400000e+01
max      3.400000e+01
Name: site_id, dtype: float64
Variable visitor_location_country_id:
count    4.958347e+06
mean     1.753405e+02
std      6.591625e+01
min      1.000000e+00
25%      1.000000e+02
50%      2.190000e+02
75%      2.190000e+02
max      2.310000e+02
Name: visitor_location_country_id, dtype: float64
Variable visitor_hist_starrating:
count    251866.000000
mean          3.374334
std           0.692519
min           

count    1.650990e+06
mean     5.776897e-02
std      2.948202e-01
min     -1.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: comp3_inv, dtype: float64
Variable comp3_rate_percent_diff:
count    472797.000000
mean         27.071559
std        1012.919911
min           2.000000
25%           7.000000
50%          11.000000
75%          18.000000
max      199266.000000
Name: comp3_rate_percent_diff, dtype: float64
Variable comp4_rate:
count    307378.000000
mean         -0.016543
std           0.578718
min          -1.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: comp4_rate, dtype: float64
Variable comp4_inv:
count    343663.000000
mean          0.096653
std           0.337165
min          -1.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: comp4_inv, dtype: float64
Variable comp4_rate_percent_diff:
count    1.310860e+0

In [33]:
corr = df_train.corr()

In [34]:
corr.sort_values('booking_bool')

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
position,-0.001505,-0.031194,0.018716,-0.014273,-0.015388,0.029850,-0.000320,-0.107717,-0.053983,0.023419,...,-0.000012,-0.016956,-0.004813,-0.003851,-0.029473,-0.008039,0.000672,-0.164992,0.005909,-0.147918
random_bool,-0.000697,0.026814,-0.037232,0.015375,0.018137,-0.030175,0.002507,-0.004763,-0.050799,-0.048390,...,0.005506,-0.008527,0.010181,0.002717,-0.012149,-0.003528,0.001481,0.005819,-0.000053,-0.088891
srch_length_of_stay,-0.001256,0.106386,-0.097225,0.077293,0.085236,-0.084679,0.001863,0.091217,-0.021135,-0.107402,...,0.002984,0.002708,-0.013568,-0.002380,0.003342,-0.014947,0.002845,0.002647,0.444524,-0.024412
srch_booking_window,-0.002946,0.168395,-0.087573,0.092211,0.111427,-0.118640,0.000222,0.074016,0.005258,-0.097656,...,-0.020707,0.020242,-0.002836,0.000433,-0.007773,-0.001552,-0.005095,0.001707,0.130837,-0.019582
comp1_inv,-0.002821,0.003661,-0.000876,-0.013398,-0.017765,-0.029908,-0.001643,-0.052368,-0.024660,-0.015947,...,0.012157,-0.006357,0.254803,-0.009372,0.007649,0.334016,0.000119,-0.007263,-0.001308,-0.007850
comp3_inv,-0.000671,0.055601,-0.035703,0.042160,-0.003338,-0.063506,0.001576,-0.011972,-0.044646,-0.103257,...,0.068471,0.028235,0.085855,0.039926,0.000744,0.299263,0.000656,-0.006723,-0.008433,-0.007438
comp7_inv,0.002822,-0.011761,-0.006172,-0.005446,0.011371,-0.061137,-0.002152,-0.028579,0.003435,-0.062500,...,0.036682,0.007573,1.000000,0.009889,-0.012090,0.093429,0.029708,-0.006915,0.020148,-0.006155
comp6_inv,0.003072,-0.005965,-0.002011,-0.008183,0.017422,-0.029468,0.006492,-0.035213,-0.014350,-0.057402,...,0.017876,0.009276,0.391647,0.015221,0.008768,0.084910,-0.002600,-0.005019,-0.002019,-0.005906
srch_adults_count,0.004007,-0.022573,0.002687,-0.011558,0.039195,0.028440,0.000213,0.006948,0.019175,0.006918,...,-0.006431,0.009343,0.002430,0.000630,-0.003230,-0.000618,-0.002645,0.004620,0.099559,-0.005376
orig_destination_distance,0.000689,0.338368,-0.056978,0.273962,0.159176,-0.365723,0.002115,0.182595,-0.050853,-0.205972,...,0.006334,0.020725,0.024992,0.014386,0.007273,-0.003557,0.002150,0.002287,0.182403,-0.003687


In [7]:
subset = df_train.head(500)
df_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,1,2.20,0.0149,5.03,26,170.74,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,1,2.20,0.0245,4.92,21,179.80,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,1,2.83,0.0125,4.39,34,602.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,-1.0,0.0,5.0,-1.0,0.0,5.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,1,2.64,0.1241,4.93,4,143.58,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [5]:
pos_matrix = subset.pivot(index='srch_id', columns='prop_id', values='position')

In [3]:
def label_score(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    return 0

df_train['score'] = df_train.apply(lambda row: label_score(row), axis=1)

In [8]:
def remove_nan(df, threshold):
    size = df.shape[0]
    

In [4]:
df_train = df_train.drop('date_time', axis=1)

In [5]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(df_train, groups=df_train['srch_id'])

X_train_inds, X_test_inds = next(gss)

train_data= df_train.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','score'])]
y_train = train_data.loc[:, train_data.columns.isin(['score'])]

groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

test_data= df_train.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['score'])]
y_test = test_data.loc[:, test_data.columns.isin(['score'])]